In [19]:
import mne 
import os
import mne 
from mne.io import  read_raw_edf
import pandas as pd
import numpy as np


In [20]:
data_folder ='data/'

In [24]:
# Set the path to the patient folder
data_files = sorted([f for f in os.listdir(data_folder) if f.startswith('chb')])
# Define the list of channels to keep
channels_to_keep = ['FP1-F7', 'FP1-F3', 'FP2-F4', 'F8-T8','FP2-F8', 'T8-P8-1', 'T8-P8-0', 'P8-O2', 'P4-O2']# 
dataframes = []  # List to store individual DataFrames

for folder_list in range(1,len(data_files[:3])):  # Assumes folders are named ch01 to ch10
        folder_name = f'chb{folder_list:02d}'  # Format the folder name as ch01, ch02, etc.
        folder_path = os.path.join(data_folder,folder_name)  # Create the full path to the folder

        # Create an empty list to store raw data
        patient_raws = []

        # Create an empty list to store file names
        file_names = []

        # Get a list of .edf files sorted by filename numbers
        edf_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.edf')])

        # Read the first .edf file to obtain the channel information
        first_file_path = os.path.join(folder_path, edf_files[0])
        first_raw = mne.io.read_raw_edf(first_file_path, preload=True, verbose='ERROR')
        first_raw.pick_channels(channels_to_keep)

        # Loop through the sorted .edf files
        for file in edf_files:
            file_path = os.path.join(folder_path, file)
            raw = mne.io.read_raw_edf(file_path, preload=True,verbose='ERROR')
            #Keep only the specified channels
            raw.pick_channels(channels_to_keep)
            raw = raw.resample(sfreq=128)

            # Set the channel information for each Raw object to match the first Raw object
            raw.info = first_raw.info
            patient_raws.append(raw)
            file_names.extend([file] * len(raw.times))

        # Concatenate the raw objects into a single patient_combined object while maintaining a continuous time axis
        patient_combined = mne.concatenate_raws(patient_raws, preload=True)
        # Print info about the combined raw object for this patient
        #print(patient_combined.info)

        df = patient_combined.to_data_frame()
        df['file_name'] = file_names
        dataframes.append(df)
# Concatenate all DataFrames into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)


Extracting EDF parameters from /Users/anabroggini/Documents/bootcamp/ai2/data/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/rc/97bvm7z97gd87g90vfhmcrq00000gn/T/ipykernel_17665/1642748961.py:22: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  first_raw = mne.io.read_raw_edf(first_file_path, preload=True)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Extracting EDF parameters from /Users/anabroggini/Documents/bootcamp/ai2/data/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/rc/97bvm7z97gd87g90vfhmcrq00000gn/T/ipykernel_17665/1642748961.py:23: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  first_raw.pick_channels(channels_to_keep)
/var/folders/rc/97bvm7z97gd87g90vfhmcrq00000gn/T/ipykernel_17665/1642748961.py:28: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

/var/folders/rc/97bvm7z97gd87g90vfhmcrq00000gn/T/ipykernel_17665/1642748961.py:30: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(channels_to_keep)


Extracting EDF parameters from /Users/anabroggini/Documents/bootcamp/ai2/data/chb02/chb02_35.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/rc/97bvm7z97gd87g90vfhmcrq00000gn/T/ipykernel_17665/1642748961.py:28: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/var/folders/rc/97bvm7z97gd87g90vfhmcrq00000gn/T/ipykernel_17665/1642748961.py:30: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(channels_to_keep)


In [27]:
final_dataframe.tail()

,time,FP1-F7,FP1-F3,FP2-F4,P4-O2,FP2-F8,F8-T8,T8-P8-0,P8-O2,T8-P8-1,file_name
34937211,63479.480469,-115.782722,-24.280219,-40.537569,14.266496,-35.614172,-3.282772,-45.686923,9.112427,-45.686923,chb02_35.edf
34937212,63479.484375,-222.561781,-49.505983,-40.070918,14.366112,-28.855441,-44.026272,-14.750472,1.683473,-14.750472,chb02_35.edf
34937213,63479.488281,-81.687330,-1.797041,-39.517940,-0.736945,-36.422067,-43.812572,-14.380229,-1.661448,-14.380229,chb02_35.edf
34937214,63479.492188,-111.241961,-17.389396,-30.708338,3.265727,-20.256353,-75.202124,6.590191,-4.620890,6.590191,chb02_35.edf
34937215,63479.496094,-40.555302,-32.466193,-8.612591,-3.440310,0.378134,1.532097,-56.972049,-10.564781,-56.972049,chb02_35.edf


In [28]:
# Loop through subfolders in the main data folder
folder_path = './data/'
all_data = []

for folder_name in os.listdir(folder_path):
    if not folder_name.startswith('chb'):
        continue
    folder_path = os.path.join(folder_path, folder_name)
    # Check if the item in the main folder is a directory (subfolder)
    if os.path.isdir(folder_path):
        # Loop through the text files in the subfolder
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt") and filename.startswith("chb"):
                file_path = os.path.join(folder_path, filename)
                
                # Extract the part of the filename before the hyphen as the "Source File" value
                source_file = filename.split("-")[0]
                
                # Initialize a list to store data for each file
                data = []
                
                # # Initialize a dictionary to hold the current data
                current_data = {"File Name": None, "Number of Seizures": None, "Seizure Start Time": np.nan, "Seizure End Time": np.nan, "Source File": source_file}
                
                # Open and read the current text file
                with open(file_path, "r") as file:
                    for line in file:
                        line = line.strip()
                        if line.startswith("File Name:"):
                            current_data = {"File Name": None, "Number of Seizures": None, "Seizure Start Time": np.nan, "Seizure End Time": np.nan, "Source File": source_file}
                            current_data["File Name"] = line.split(": ", 1)[1]
                        elif line.startswith("Number of Seizures in File:"):
                            current_data["Number of Seizures"] = line.split(": ",1)[1]
                            if current_data["Number of Seizures"] == 0:
                                data.append(current_data.copy())
                        elif line.startswith("Seizure Start Time:"):
                            current_data["Seizure Start Time"] = int(line.split(": ", 1)[1].split()[0])
                        elif line.startswith("Seizure End Time:"):
                            current_data["Seizure End Time"] = int(line.split(": ", 1)[1].split()[0])
                            data.append(current_data.copy())
                
                all_data.extend(data)

# Create a DataFrame from the collected data
seizure_seq = pd.DataFrame(all_data)
seizure_seq = seizure_seq.rename(columns={"File Name": "file_name"})

# Display the DataFrame
print(seizure_seq)

      file_name Number of Seizures  Seizure Start Time  Seizure End Time  \
0  chb03_01.edf                  1                 362               414   
1  chb03_02.edf                  1                 731               796   
2  chb03_03.edf                  1                 432               501   
3  chb03_04.edf                  1                2162              2214   
4  chb03_34.edf                  1                1982              2029   
5  chb03_35.edf                  1                2592              2656   
6  chb03_36.edf                  1                1725              1778   

  Source File  
0       chb03  
1       chb03  
2       chb03  
3       chb03  
4       chb03  
5       chb03  
6       chb03  


In [29]:
# Define a function to check if a recording occurred during a seizure
def during_seizure(file_name, time):
    seizure_data = seizure_seq[seizure_seq["file_name"] == file_name]
    if not seizure_data.empty:
        return any((time >= seizure_data["Seizure Start Time"]) & (time <= seizure_seq["Seizure End Time"]))
    else:
        return False

In [30]:
final_dataframe["seizure"] = final_dataframe.apply(lambda row: during_seizure(row["file_name"], row["time"]), axis=1)

In [18]:
final_dataframe.tail()

,time,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,...,T7-FT9,FT9-FT10,FT10-T8,T8-P8-1,file_name,Number of Seizures,Seizure Start Time,Seizure End Time,Source File,seizure_sequence
34937211,63479.480469,-115.782722,134.907221,-46.812467,1.358508,-24.280219,7.271520,-37.914306,28.991814,-40.537569,...,-24.796244,-6.904201,-8.270946,-45.686923,chb02_35.edf,NaN,NaN,NaN,NaN,False
34937212,63479.484375,-222.561781,194.940960,-50.500316,-15.615978,-49.505983,-11.458245,-35.386708,3.300393,-40.070918,...,-61.071168,9.364014,-26.803702,-14.750472,chb02_35.edf,NaN,NaN,NaN,NaN,False
34937213,63479.488281,-81.687330,49.461055,48.180296,-22.157032,-1.797041,33.254509,-40.797284,3.367640,-39.517940,...,45.751252,14.268259,-55.746283,-14.380229,chb02_35.edf,NaN,NaN,NaN,NaN,False
34937214,63479.492188,-111.241961,94.053730,7.131500,-4.767848,-17.389396,33.662479,-40.187919,10.268091,-30.708338,...,-14.590857,15.875116,-48.629292,6.590191,chb02_35.edf,NaN,NaN,NaN,NaN,False
34937215,63479.496094,-40.555302,45.929406,-14.705387,9.535962,-32.466193,43.082463,-40.838351,30.218679,-8.612591,...,20.085840,1.748631,-14.146080,-56.972049,chb02_35.edf,NaN,NaN,NaN,NaN,False
